In [7]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`

import $ivy.$                                  

In [8]:
import org.apache.spark.sql._

import org.apache.spark.sql._

In [9]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [10]:
import $ivy.`sh.almond::almond-spark:0.6.0`

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                              


spark: SparkSession = org.apache.spark.sql.SparkSession@4b6889db

In [11]:
def sc = spark.sparkContext

defined function sc

In [12]:
val rdd = sc.parallelize(1 to 100000000, 100)

rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at cmd11.sc:1

In [13]:
import spark.implicits._

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this);

import spark.implicits._



In [14]:
var trainData = spark.sqlContext.read
                    .format("com.databricks.spark.csv")
                        .option("header", "true")
                        .option("mode", "DROPMALFORMED")
                        .option("escape", "\"")
                            .load("train.csv")
                                .filter($"text".isNotNull)
                                .filter($"target".isNotNull)

var testData = spark.sqlContext.read
                    .format("com.databricks.spark.csv")
                        .option("header", "true")
                        .option("mode", "DROPMALFORMED")
                        .option("escape", "\"")
                            .load("test.csv")
                                .filter($"text".isNotNull)

load at cmd13.sc:6

1 / 1

load at cmd13.sc:15

1 / 1

trainData: Dataset[Row] = [id: string, keyword: string ... 3 more fields]
testData: DataFrame = [id: string, keyword: string ... 2 more fields]

In [15]:
trainData.columns
testData.columns

res14_0: Array[String] = Array("id", "keyword", "location", "text", "target")
res14_1: Array[String] = Array("id", "keyword", "location", "text")

In [16]:
implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20, truncate: Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map { row =>
      row.toSeq.map { cell =>
        val str = cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }
        if (truncate > 0 && str.length > truncate) {
          // do not show ellipses for strings shorter than 4 characters.
          if (truncate < 4) str.substring(0, truncate)
          else str.substring(0, truncate - 3) + "..."
        } else {
          str
        }
      }: Seq[String]
    }

    publish.html(s"""
      <table class="table">
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
      </table>""")
  }
}

defined class RichDF

In [17]:
trainData.showHTML(5)

take at cmd15.sc:4

1 / 1

id,keyword,location,text,target
1,null,null,Our Deeds are the...,1
4,null,null,Forest fire near ...,1
5,null,null,All residents ask...,1
6,null,null,"13,000 people rec...",1
7,null,null,Just got sent thi...,1


In [18]:
testData.showHTML(5)

take at cmd15.sc:4

1 / 1

id,keyword,location,text
0,null,null,Just happened a t...
2,null,null,Heard about #eart...
3,null,null,there is a forest...
9,null,null,Apocalypse lighti...
11,null,null,Typhoon Soudelor ...


In [19]:
trainData.describe().showHTML()

describe at cmd18.sc:1

1 / 1

describe at cmd18.sc:1

1 / 1

summary,id,keyword,location,text,target
count,7176,7115,4771,7176,7176
mean,5434.829152731327,null,8412.318181818182,null,0.42934782608695654
stddev,3142.3697249082106,null,24663.97600393154,null,0.49501759344035795
min,1,ablaze,,! Residents Retur...,0
max,9998,wrecked,åø\_(?)_/åø,åÈMGN-AFRICAå¨ pi...,1


In [20]:
testData.describe().showHTML()

describe at cmd19.sc:1

1 / 1

describe at cmd19.sc:1

1 / 1

summary,id,keyword,location,text
count,3261,3235,2156,3261
mean,5425.076050291322,null,98593.33333333333,null
stddev,3146.111609080526,null,225255.45173927903,null
min,0,ablaze,,! Sex-themed e-bo...
max,9999,wrecked,å©hicago,Û÷SexistÛª peer...


In [22]:
// only Scala 2.11
import $ivy.`org.vegas-viz::vegas:0.3.11`

import vegas._
import vegas.data.External._

Vegas("There are more tweets with target 0 (no disaster) than target 1 (disaster tweets)").
    withData(Seq(
        Map("value" -> "0", "count" ->
            trainData.filter("target = 0").describe().select("target").first.getString(0).toInt),
        Map("value" -> "1", "count" ->
            trainData.filter("target = 1").describe().select("target").first.getString(0).toInt)
    )).
    encodeX("value", Ordinal).
    encodeY("count", Quantitative).
    mark(Bar).
    show

describe at cmd21.sc:9

1 / 1

describe at cmd21.sc:9

1 / 1

describe at cmd21.sc:11

1 / 1

describe at cmd21.sc:11

1 / 1

<iframe id="frame-vegas-fb289758-f756-47ac-9d82-737f982622ff" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-fb289758-f756-47ac-9d82-737f982622ff'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "value",
 "type" : "ordinal"
 },
 "y" : {
 "field" : "count",
 "type" : "quantitative"
 }
 },
 "description" : "There are more tweets with target 0 (no disaster) than target 1 (disaster tweets)",
 "data" : {
 "values" : [
 {
 "value" : "0",
 "count" : 4095
 },
 {
 "value" : "1",
 "count" : 3081
 }
 ]
 }
}
 }
 vg.embed("#vegas-fb289758-f756-47ac-9d82-737f982622ff", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

import $ivy.$                            


import vegas._

import vegas.data.External._



In [23]:
import org.apache.spark.sql.functions._

var cleanTrainData = trainData.withColumn("text", regexp_replace(col("text"), "n\'t", " not"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'re", " are"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'s", " is"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'d", " would"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'ll", " will"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'t", " not"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'ve", " have"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\'m", " am"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "won\'t", "will not"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "can\'t", "can not"))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "<.*?>+", ""))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\\W", " "))
cleanTrainData = cleanTrainData.withColumn("text", regexp_replace(col("text"), "\n", ""))
cleanTrainData = cleanTrainData.withColumn("text", lower(col("text")))

cleanTrainData.showHTML(10)

take at cmd15.sc:4

1 / 1

id,keyword,location,text,target
1,null,null,our deeds are the...,1
4,null,null,forest fire near ...,1
5,null,null,all residents ask...,1
6,null,null,13 000 people rec...,1
7,null,null,just got sent thi...,1
8,null,null,rockyfire update...,1
10,null,null,flood disaster ...,1
13,null,null,i am on top of th...,1
14,null,null,there is an emerg...,1
15,null,null,i am afraid that ...,1


import org.apache.spark.sql.functions._


cleanTrainData: DataFrame = [id: string, keyword: string ... 3 more fields]

In [24]:
import $ivy.`org.apache.spark::spark-mllib:2.4.3`

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.PipelineStage
import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.feature.HashingTF
import org.apache.spark.ml.feature.IDF
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.StringIndexer

import $ivy.$                                    


import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.PipelineStage

import org.apache.spark.ml.classification.GBTClassifier

import org.apache.spark.ml.feature.HashingTF

import org.apache.spark.ml.feature.IDF

import org.apache.spark.ml.feature.RegexTokenizer

import org.apache.spark.ml.feature.StopWordsRemover

import org.apache.spark.ml.feature.StringIndexer

In [25]:
import $ivy.`com.github.master:spark-stemming_2.10:0.2.1`

import org.apache.spark.mllib.feature.Stemmer

import $ivy.$                                            


import org.apache.spark.mllib.feature.Stemmer

In [28]:
val regexTokenizer = new RegexTokenizer()
                                    .setInputCol("text")
                                    .setOutputCol("words")
                                    .setPattern("\\W")

val stopWordsRemover = new StopWordsRemover()
                                    .setInputCol("words")
                                    .setOutputCol("removed")

// only Scala 2.11
val stemmer = new Stemmer()
                                    .setInputCol("removed")
                                    .setOutputCol("stemmed")
                                    .setLanguage("English")

val hashingTF = new HashingTF()
                                    .setInputCol("stemmed")
                                    .setNumFeatures(3000)
                                    .setOutputCol("rawFeatures")

val idf = new IDF()
                                    .setInputCol("rawFeatures")
                                    .setOutputCol("features")

val stringIndexer = new StringIndexer()
                                    .setInputCol("target")
                                    .setOutputCol("indexedLabel")

val gbt = new GBTClassifier()
                                    .setLabelCol("indexedLabel")
                                    .setFeaturesCol("features")
                                    .setPredictionCol("predictionTarget")
                                    .setMaxIter(30)

regexTokenizer: RegexTokenizer = regexTok_4401742f2c50
stopWordsRemover: StopWordsRemover = stopWords_fd7831553d9a
stemmer: Stemmer = stemmer_4dd04471ed37
hashingTF: HashingTF = hashingTF_348821016f8a
idf: IDF = idf_abcf437516ac
stringIndexer: StringIndexer = strIdx_c6bf2c7d6324
gbt: GBTClassifier = gbtc_0ed24fb6da48

In [29]:
val pipeline = new Pipeline()
                            .setStages(
                                Array(
                                    regexTokenizer, stopWordsRemover, stemmer, hashingTF, idf, stringIndexer, gbt
                                )
                            )

pipeline: Pipeline = pipeline_3d4163ae470f

In [30]:
var cleanTestData = testData.withColumn("text", regexp_replace(col("text"), "n\'t", " not"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'re", " are"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'s", " is"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'d", " would"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'ll", " will"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'t", " not"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'ve", " have"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\'m", " am"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "won\'t", "will not"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "can\'t", "can not"))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "<.*?>+", ""))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\\W", " "))
cleanTestData = cleanTestData.withColumn("text", regexp_replace(col("text"), "\n", ""))
cleanTestData = cleanTestData.withColumn("text", lower(col("text")))

cleanTestData: DataFrame = [id: string, keyword: string ... 2 more fields]

In [31]:
val pipelineModel = pipeline.fit(cleanTrainData)
val fullPredictions = pipelineModel.transform(cleanTestData)

treeAggregate at IDF.scala:54

1 / 1

countByValue at StringIndexer.scala:140

1 / 1

countByValue at StringIndexer.scala:140

...

mapPartitions at RandomForest.scala:538

1 / 1

collectAsMap at RandomForest.scala:567

1 / 1

pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_3d4163ae470f
fullPredictions: DataFrame = [id: string, keyword: string ... 10 more fields]

In [32]:
import org.apache.spark.sql.types.IntegerType

var result = fullPredictions
                    .select("id", "predictionTarget")
                        .withColumn("target", fullPredictions("predictionTarget").cast(IntegerType))
                            .drop("predictionTarget")

result.showHTML(5)

result.write
        .format("com.databricks.spark.csv")
            .option("header", "true")
                .save("result.csv")

take at cmd15.sc:4

1 / 1

20/11/22 17:31:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
20/11/22 17:31:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


id,target
0,0
2,0
3,1
9,0
11,1


save at cmd31.sc:15

1 / 1

import org.apache.spark.sql.types.IntegerType


result: DataFrame = [id: string, target: int]

In [33]:
val mySubmission = spark.sqlContext.read
                    .format("csv")
                        .option("header", "true")
                        .option("mode", "DROPMALFORMED")
                        .option("escape", "\"")
                            .csv("result.csv/*.csv")

mySubmission.describe().showHTML()

csv at cmd32.sc:6

1 / 1

describe at cmd32.sc:8

1 / 1

describe at cmd32.sc:8

1 / 1

summary,id,target
count,3261,3261
mean,5425.076050291322,0.2946948788715118
stddev,3146.111609080526,0.45597539938904524
min,0,0
max,9999,1


mySubmission: DataFrame = [id: string, target: string]

In [34]:
val sampleSubmission = spark.sqlContext.read
                    .format("csv")
                        .option("header", "true")
                        .option("mode", "DROPMALFORMED")
                        .option("escape", "\"")
                            .csv("sample_submission.csv")
                                .select("id")

sampleSubmission.describe().showHTML(5)

csv at cmd33.sc:6

1 / 1

describe at cmd33.sc:9

1 / 1

describe at cmd33.sc:9

1 / 1

summary,id
count,3263
mean,5427.15292675452
stddev,3146.4272214965617
min,0
max,9999


sampleSubmission: DataFrame = [id: string]

In [35]:
val df_result = mySubmission.as("dfresult")
val df_sampleSubmission = sampleSubmission.as("dfsamplesubmission")


val joinedDF = df_result
                        .join(df_sampleSubmission, 
                              col("dfsamplesubmission.id") === col("dfresult.id"),
                              "right")                    

var resultDF = joinedDF
                        .select(col("dfsamplesubmission.id"),
                                when(col("dfresult.id").isNull, lit(0))
                                                        .otherwise(col("target"))
                                                            .as("target"))

resultDF.describe().showHTML()

run at ThreadPoolExecutor.java:1149

1 / 1

describe at cmd34.sc:16

1 / 1

describe at cmd34.sc:16

1 / 1

summary,id,target
count,3263,3263
mean,5427.15292675452,0.2945142506895495
stddev,3146.4272214965617,0.45589395982167097
min,0,0
max,9999,1


df_result: Dataset[Row] = [id: string, target: string]
df_sampleSubmission: Dataset[Row] = [id: string]
joinedDF: DataFrame = [id: string, target: string ... 1 more field]
resultDF: DataFrame = [id: string, target: string]

In [36]:
resultDF.repartition(1).write
                                .format("com.databricks.spark.csv")
                                   .option("header", "true")
                                       .save(sys.env("HOME") + "/Documents/disasterTweets/")

run at ThreadPoolExecutor.java:1149

1 / 1

save at cmd35.sc:4

1 / 1

save at cmd35.sc:4

1 / 1